In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
from pmdarima.arima import auto_arima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot

In [2]:
confirmed_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
confirmed_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,299,349,367,423,444,484,521,555,607,665
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,333,361,377,383,400,409,416,433,446,467
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1251,1320,1423,1468,1572,1666,1761,1825,1914,1983
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,466,501,525,545,564,583,601,601,638,646
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,10,14,16,17,19,19,19,19,19,19


In [3]:
deaths_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
deaths_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,7,7,11,14,14,15,15,18,18,21
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,20,20,21,22,22,23,23,23,23,23
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,130,152,173,193,205,235,256,275,293,313
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,17,18,21,22,23,25,26,26,29,29
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [4]:
recoveries_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
recoveries_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,10,15,18,18,29,32,32,32,32,32
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,99,104,116,131,154,165,182,197,217,232
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,90,90,90,113,237,347,405,460,591,601
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,21,26,31,39,52,58,71,71,128,128
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,4,4,4


In [5]:
colnames = confirmed_cases.columns.values
colnames

array(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20',
       '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20',
       '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20',
       '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20',
       '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20',
       '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20',
       '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20',
       '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20',
       '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20',
       '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20',
       '4

In [6]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colnames[4]:colnames[-1]]

In [7]:
time_date = confirmed.columns.values

In [8]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

# Predictions for days ahead

In [9]:
#pred_days_ahead = 2
#pred_days_ahead = 7
pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day = np.array([i for i in range(len(time_date)+pred_days_ahead+1)]).reshape(-1, 1)

In [10]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day))]

In [11]:
countries = ['Greece', 'Italy', 'Spain', 'Turkey','Cyprus', 'Switzerland']

In [12]:
prediction_models = dict()
prediction_models_confirmed = {'SVR': 0}
prediction_models_deaths = {'SVR': 0}
prediction_models_recoveries = {'SVR': 0}


In [13]:
# Prediction Models

from sklearn import model_selection
from sklearn.svm import SVR
def SVM_regression(train_x, train_y, test_x, test_y, X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
   # c = [0.01, 0.1, 1]
   # gamma = [0.01, 0.1, 1]
   #epsilon = [0.01, 0.1, 1]
   #shrinking = [True, False]
    #degree = [3, 4, 5, 6]
  #grid_parameters = {'C': c, 'gamma' : gamma ,'epsilon': epsilon, 'shrinking' : shrinking, 'degree':degree} 
   #regressor = RandomizedSearchCV(SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1, C=0.1), grid_parameters, scoring='neg_mean_squared_error', 
    #                               cv=5)
    
    if (subpopulation == 'confirmed'):
         degree = 3
    elif (subpopulation == 'deaths'):
        degree = 4
    else :
        degree = 5


    regressor = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=degree, C=0.1)
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['SVR'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['SVR'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['SVR'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions) 
    

In [14]:
from sklearn.ensemble import RandomForestRegressor
def RF_regression(train_x, train_y, test_x, test_y,X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    grid_parameters = {'n_estimators': n_estimators}
#    regressor = RandomizedSearchCV(RandomForestRegressor(), grid_parameters, scoring='neg_mean_squared_error', 
#                                   cv=3)
    
    regressor = RandomForestRegressor(n_estimators = 200, random_state = 42) 
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)


In [15]:
# Gradient Boosted Regression Trees (GBRT) 
from sklearn.ensemble import GradientBoostingRegressor

def Grad_Boost_regression(train_x, train_y, test_x, test_y,X_test,subpopulation, X_train, Y_train):
   
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    loss = ['ls','lad','huber']
    grid_parameters = {'n_estimators': n_estimators}
  #  regressor = RandomizedSearchCV(GradientBoostingRegressor(learning_rate=0.1), grid_parameters, scoring='neg_mean_squared_error', 
 #                                  cv=3)
    
    regressor =  GradientBoostingRegressor(n_estimators=500, learning_rate=0.1,
                                            max_depth=1, random_state=0, loss='huber')
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)



In [16]:
#check if a timeseries is stationary
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()     
    rolstd = timeseries.rolling(12).std()

    #Plot rolling statistics:
    #orig = plt.plot(timeseries, color='blue',label='Original')
    #mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    #std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    #plt.legend(loc='best')
    #plt.title('Rolling Mean & Standard Deviation')
    #plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    #print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput
    

In [17]:
def pred_with_arima(timeseries,n_days_ahead):
    
    #find the optimal d 
    diff=timeseries-timeseries.shift(1)
    for d in range(1,10):
        if (test_stationarity(diff.dropna())['p-value']<0.1):
            #print (d) 
            break
        else: 
            d=d+1 
            diff=diff-diff.shift(1)
    #use the arima model to make predictions, use original timeseries with optimal d:
    
    model = pm.auto_arima(timeseries, start_p=0, start_q=0,
                             max_p=4, max_q=4,
                              seasonal=False,
                             d=d, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=False)
    forecasts=model.predict(n_days_ahead)
    return forecasts #returns an array

In [18]:
def Arima_Analysis(region,pred_days_ahead):
   #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region)#this is a dataframe with dates and cases for Greece.
    #df['datetime'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    #df.drop(['date'], axis=1, inplace=True)
    #df.plot( style='k.')
    #pyplot.show()
    df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis. 
    pred_ts=pred_with_arima(df['cases'],pred_days_ahead).tolist() #this is an array converted to a list
    return pred_ts

In [19]:
import operator
def subpopulation_Analysis(cases,country, all_days,predictions_ahead,subpopulation):
    
    pred  = list()
    region = cases.loc[cases['Country/Region'] == country]
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)

    
    train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    pred_arima=Arima_Analysis(reg,pred_days_ahead)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]  
        else:
            best_pred=pred_arima
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
        else:
            best_pred=pred_arima
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
        else:
            best_pred=pred_arima
        
    

    best_pred_ahead = best_pred[-1]
    return best_pred_ahead, region, best_pred

In [20]:
#pred_file = str(2)+"day_prediction_" +  date.today().strftime('%Y-%m-%d') + ".csv"
#pred_file = str(7)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file = str(30)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file

'30day_prediction_2020-04-14.csv'

In [21]:
columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0
for co in range(len(countries)):
    
    total_predictions = pd.DataFrame(columns = columns)
    k = k+1
   
    best_pred_confirmed, region_con, best_pred_con = subpopulation_Analysis(confirmed_cases,countries[co],
                                                 all_days,predictions_ahead,"confirmed")
  
    best_pred_deaths, region_d, best_pred_d = subpopulation_Analysis(deaths_cases,countries[co],
                                              all_days,predictions_ahead,"deaths")
    
    best_pred_recoveries, region_r, best_pred_r = subpopulation_Analysis(recoveries_cases,countries[co],
                                                  all_days,predictions_ahead,"recoveries")
    
    # Mortality rate
    mort_pred  = best_pred_deaths/best_pred_confirmed
    
    total_predictions = total_predictions.append({'Province/State': '',
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed) ,
                                                'low95N':'',
                                                'high95N': '',
                                                'R': int(best_pred_recoveries) ,
                                                'low95R' : '',
                                                'high95R' : '',
                                                'D': int(best_pred_deaths) ,
                                                'low95D' : '',
                                                'high95D': '',
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred,
                                                'low95M': '',
                                                'high95M': '' ,
                                                'C': '' ,
                                                'low95C': '' ,
                                                'high95C': '' },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions.to_csv(pred_file, sep=',',index = False,  mode='a', header = header)
    

Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=466.482, BIC=470.139, Time=0.023 seconds
Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=443.711, BIC=449.197, Time=0.064 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=444.573, BIC=451.887, Time=0.172 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=439.931, BIC=449.074, Time=0.270 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=441.552, BIC=452.524, Time=0.284 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=441.648, BIC=447.134, Time=0.088 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=442.703, BIC=450.018, Time=0.097 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=444.513, BIC=453.656, Time=0.274 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=441.660, BIC=452.632, Time=0.318 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=442.301, BIC=455.101, Time=0.591 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (const

Fit ARIMA: (1, 4, 3)x(0, 0, 0, 0) (constant=True); AIC=604.939, BIC=616.290, Time=0.660 seconds
Fit ARIMA: (1, 4, 4)x(0, 0, 0, 0) (constant=True); AIC=612.025, BIC=625.268, Time=0.584 seconds
Fit ARIMA: (2, 4, 0)x(0, 0, 0, 0) (constant=True); AIC=653.807, BIC=661.374, Time=0.122 seconds
Fit ARIMA: (2, 4, 1)x(0, 0, 0, 0) (constant=True); AIC=618.936, BIC=628.395, Time=0.229 seconds
Fit ARIMA: (2, 4, 2)x(0, 0, 0, 0) (constant=True); AIC=607.288, BIC=618.639, Time=0.394 seconds
Fit ARIMA: (2, 4, 3)x(0, 0, 0, 0) (constant=True); AIC=605.884, BIC=619.127, Time=0.707 seconds
Fit ARIMA: (3, 4, 0)x(0, 0, 0, 0) (constant=True); AIC=639.895, BIC=649.355, Time=0.159 seconds
Fit ARIMA: (3, 4, 1)x(0, 0, 0, 0) (constant=True); AIC=614.023, BIC=625.374, Time=0.323 seconds
Fit ARIMA: (3, 4, 2)x(0, 0, 0, 0) (constant=True); AIC=609.263, BIC=622.506, Time=0.706 seconds
Fit ARIMA: (4, 4, 0)x(0, 0, 0, 0) (constant=True); AIC=628.799, BIC=640.150, Time=0.269 seconds
Fit ARIMA: (4, 4, 1)x(0, 0, 0, 0) (const

Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=882.361, BIC=896.784, Time=0.990 seconds
Fit ARIMA: (4, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=881.025, BIC=893.388, Time=0.626 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=882.366, BIC=896.790, Time=0.903 seconds
Total fit time: 10.296 seconds
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=475.312, BIC=478.244, Time=0.023 seconds
Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=476.235, BIC=480.632, Time=0.142 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=477.520, BIC=483.383, Time=0.354 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=478.436, BIC=485.764, Time=0.727 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=475.918, BIC=484.712, Time=0.737 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=476.631, BIC=481.028, Time=0.097 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=478.795, BIC=484.658, Time=0.209 seconds
Fit ARIMA

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=60.818, BIC=65.182, Time=0.292 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=61.890, BIC=67.345, Time=0.456 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=62.829, BIC=69.375, Time=0.695 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=62.778, BIC=70.416, Time=1.144 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=65.020, BIC=69.384, Time=0.083 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=60.683, BIC=66.138, Time=0.592 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=63.259, BIC=69.805, Time=0.571 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=61.347, BIC=68.985, Time=0.750 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=63.653, BIC=69.109, Time=0.128 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=60.664, BIC=67.210, Time=0.535 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=62.66

Fit ARIMA: (3, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=598.572, BIC=608.705, Time=0.589 seconds
Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=597.920, BIC=609.742, Time=0.863 seconds
Fit ARIMA: (4, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=591.628, BIC=601.762, Time=0.315 seconds
Fit ARIMA: (4, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=592.718, BIC=604.540, Time=0.398 seconds
Total fit time: 8.461 seconds
